In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
import numpy as np
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
loader = PyPDFDirectoryLoader("./us_census")

documents  = loader.load()

docs = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap=200).split_documents(documents)
docs[0]

Document(metadata={'source': 'us_census\\acsbr-015.pdf', 'page': 0}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015\nIssued September 2023\nDouglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to \nhealth coverage. For example, between 2021 and 2022, \nthe labor market continued to improve, which may \nhave affected private coverage in the United States \nduring that time.1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under \nthe Continuous Enrollment Provision.2 The American \nRescue Plan (ARP) enhanced Marketplace premium \nsubsidies for those with incomes above 400 percent \nof the poverty level as well as for unemployed people.3\nIn addition to national policies, individual states and \nthe District of Columbia can affect 

In [3]:
len(docs)

316

In [7]:
embedding = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={"device":'cpu'},
    encode_kwargs = {'normalize_embeddings':True}
)

c:\Users\Appsb\Desktop\LangChain\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Appsb\Desktop\LangChain\myenv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Appsb\.cache\huggingface\hub\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activa

In [8]:
np.array(embedding.embed_query(docs[0].page_content))

array([-0.07903481, -0.01134112, -0.02312098,  0.02844463,  0.05053343,
        0.05317828, -0.01907791,  0.03456023, -0.1021137 , -0.02915701,
        0.0852426 ,  0.05650729, -0.02545437, -0.0330849 , -0.00635735,
        0.04090865, -0.00628106,  0.00356741, -0.03854128,  0.03667685,
       -0.04289803,  0.03425249, -0.03116902, -0.0379373 ,  0.01728393,
        0.01214923,  0.00653118,  0.01463566, -0.05529052, -0.15320708,
        0.00730848,  0.03202941, -0.04701128, -0.01595975,  0.01874448,
        0.02642936, -0.02306381,  0.08438039,  0.04182487,  0.05278176,
       -0.03057603,  0.0156426 , -0.01689073,  0.00529407, -0.02417434,
        0.00412997, -0.01889939, -0.00150625, -0.0083694 , -0.03390061,
        0.03515957, -0.00553132,  0.04910939,  0.05971861,  0.05615965,
       -0.05105154,  0.01475135, -0.01849959, -0.03284641,  0.03576629,
        0.04947707, -0.00938882, -0.26202115,  0.09750333,  0.01715692,
        0.04781386, -0.00556319, -0.00298308, -0.02207354, -0.04

In [9]:
vectorstore = FAISS.from_documents(docs[:120],embedding)

In [10]:
query = "What is health insurance coverage"
relevant_documents = vectorstore.similarity_search(query)
print(relevant_documents[0].page_content)


2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [11]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000241A00267B0> search_kwargs={'k': 3}


In [12]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")


In [ ]:
from langchain_community.llms import HuggingFaceHub
hf = HuggingFaceHub(
    repo_id = "mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.1,"max_length":500}
)
hf.invoke(query)

In [ ]:
#Hugging Face models can be run locally through the HuggingFacePipeline class.
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-v0.1",
    task="text-generation",
    pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
)

llm = hf 
llm.invoke(query)

In [ ]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [ ]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [ ]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [ ]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [ ]:
result = retrievalQA.invoke({"query": query})
print(result['result'])